In [1]:
import os
import time
import math
from contextlib import nullcontext

import numpy as np
import pandas as pd
from tqdm import tqdm

import torch
from torch.utils.data import Dataset, DataLoader

from nanogpt_model import GPTConfig, GPT

In [2]:
out_dir = "out"
eval_interval = 100
log_interval = 1
eval_only = False # if True, script exits right after the first eval
always_save_checkpoint = False # if True, always save a checkpoint after each eval
init_from = "resume" # "scratch" or "resume"

# data
gradient_accumulation_steps = 4 # used to simulate larger batch sizes
batch_size = 8 # if gradient_accumulation_steps > 1, this is the micro-batch size
block_size = 512
vocab_size = 217

# model
n_layer = 8
n_head = 8
n_embd = 256
dropout = 0.0 # for pretraining 0 is good, for finetuning 0.1
bias = False # do we use bias inside LayerNorm and Linear layers?

# adamw optimizer
learning_rate = 6e-4 #5e-5 for finetuning # max learning rate
max_iters = 600000 # total number of training iterations
weight_decay = 1e-1
beta1 = 0.9
beta2 = 0.95
grad_clip = 1.0 # clip gradients at this value, or disable if == 0.0

# learning rate decay settings
decay_lr = True # whether to decay the learning rate
warmup_iters = 2000 # how many steps to warm up for
lr_decay_iters = 600000 # should be ~= max_iters per Chinchilla
min_lr = 6e-5 #5e-6 for finetuning # minimum learning rate, should be ~= learning_rate/10 per Chinchilla

device = "cuda"
dtype = "bfloat16" if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else "float16" # "float32", "bfloat16", or "float16", the latter will auto implement a GradScaler
compile_model = False # use PyTorch 2.0 to compile the model to be faster

In [3]:
os.makedirs(out_dir, exist_ok=True)

torch.backends.cuda.matmul.allow_tf32 = True # allow tf32 on matmul
torch.backends.cudnn.allow_tf32 = True # allow tf32 on cudnn
device_type = "cuda" if "cuda" in device else "cpu" # for later use in torch.autocast
# note: float16 data type will automatically use a GradScaler
ptdtype = {"float32": torch.float32, "bfloat16": torch.bfloat16, "float16": torch.float16}[dtype]
ctx = nullcontext() if device_type == "cpu" else torch.amp.autocast(device_type=device_type, dtype=ptdtype)

In [ ]:
model_args = dict(n_layer=n_layer, n_head=n_head, n_embd=n_embd, block_size=block_size,
                  bias=bias, vocab_size=vocab_size, dropout=dropout) # start with model_args from command line
if init_from == "scratch":
    print("Initializing a new model from scratch")

    gptconf = GPTConfig(**model_args)
    model = GPT(gptconf)
    iter_num = 0
    best_val_loss = 1e9
elif init_from == "resume":
    print(f"Resuming model from {out_dir}")
    ckpt_path = os.path.join(out_dir, "ckpt_beethoven.pt") # ckpt_pre_trained.pt
    checkpoint = torch.load(ckpt_path, map_location=device)
    checkpoint_model_args = checkpoint["model_args"]
    for k in ["n_layer", "n_head", "n_embd", "block_size", "bias", "vocab_size"]:
        model_args[k] = checkpoint_model_args[k]

    gptconf = GPTConfig(**model_args)
    model = GPT(gptconf)
    state_dict = checkpoint["model"]

    unwanted_prefix = "_orig_mod."
    for k,v in list(state_dict.items()):
        if k.startswith(unwanted_prefix):
            state_dict[k[len(unwanted_prefix):]] = state_dict.pop(k)
    model.load_state_dict(state_dict)
    iter_num = checkpoint["iter_num"]
    best_val_loss = checkpoint["best_val_loss"]

model.to(device)

In [ ]:
scaler = torch.cuda.amp.GradScaler(enabled=(dtype == "float16"))

# optimizer
optimizer = model.configure_optimizers(weight_decay, learning_rate, (beta1, beta2), device_type)
if init_from == "resume":
    optimizer.load_state_dict(checkpoint["optimizer"])
checkpoint = None

if compile_model:
    print("compiling the model... (takes a ~minute)")
    unoptimized_model = model
    model = torch.compile(model) # requires PyTorch 2.0

In [ ]:
midi_df = pd.read_csv("midi_dataset_512.csv")
midi_df_len = len(midi_df)
print(f"Total chunks: {midi_df_len}")
midi_df = midi_df.sample(frac=1).reset_index(drop=True) # Shuffle and drop indexes
midi_df.head()

In [ ]:
midi_tokens_dataset = np.load(os.path.join("midi_tokens_dataset_512.npy"))
midi_tokens_dataset = torch.tensor(midi_tokens_dataset, dtype=torch.int64)
print("midi_tokens_dataset shape", midi_tokens_dataset.shape)

In [ ]:
unique_tokens = torch.from_numpy(np.load("unique_tokens.npy"))
vocab_size = len(unique_tokens)
print("vocab_size", vocab_size)
unique_tokens

In [ ]:
tokens_mapping = {unique_tokens[i].item(): i for i in range(len(unique_tokens))}
tokens_unmapping = {i: unique_tokens[i].item() for i in range(len(unique_tokens))}
print(tokens_mapping)
print(tokens_unmapping)

In [8]:
def map_tokens(tokens, tokens_mapping):
    mapped_tokens = torch.zeros_like(tokens)
    for i in range(len(tokens)):
        mapped_tokens[i] = tokens_mapping[tokens[i].item()]
    return mapped_tokens

def unmap_tokens(mapped_tokens, tokens_unmapping):
    unmapped_tokens = torch.zeros_like(mapped_tokens)
    for i in range(len(mapped_tokens)):
        unmapped_tokens[i] = tokens_unmapping[mapped_tokens[i].item()]
    return unmapped_tokens

In [ ]:
train_set_len = round(midi_df_len*0.99)
print(f"Training set len: {train_set_len}, Validation set len: {midi_df_len - train_set_len}.")
midi_df_train = midi_df.iloc[:train_set_len].reset_index(drop=True)
midi_df_val = midi_df.iloc[train_set_len:].reset_index(drop=True)

In [15]:
class MidiDataset(Dataset):
    def __init__(self, midi_tokens_dataset, midi_df):
        self.midi_tokens_dataset = midi_tokens_dataset[midi_df["chunk_idx"].tolist()]

    def __len__(self):
        return self.midi_tokens_dataset.shape[0]

    def __getitem__(self, idx):
        chunk_tokens = self.midi_tokens_dataset[idx]
        chunk_tokens = map_tokens(chunk_tokens, tokens_mapping)
        x = chunk_tokens[:-1]
        y = chunk_tokens[1:]
        return x, y

In [16]:
train_dataset = MidiDataset(midi_tokens_dataset, midi_df_train)
val_dataset = MidiDataset(midi_tokens_dataset, midi_df_val)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

In [23]:
@torch.no_grad()
def estimate_val_loss():
    model.eval()
    losses = torch.zeros(len(val_loader))
    print("Calculating validation loss...")
    for i, (X, Y) in enumerate(tqdm(val_loader)):
        X = X.to(device)
        Y = Y.to(device)
        """ if i >= eval_iters:
            break """
        with ctx:
            _, loss = model(X, Y)
        losses[i] = loss.item()
    model.train()
    return losses.mean().item()

# learning rate decay scheduler (cosine with warmup)
def get_lr(it):
    # 1) linear warmup for warmup_iters steps
    if it < warmup_iters:
        return learning_rate * (it + 1) / (warmup_iters + 1)
    # 2) if it > lr_decay_iters, return min learning rate
    if it > lr_decay_iters:
        return min_lr
    # 3) in between, use cosine decay down to min learning rate
    decay_ratio = (it - warmup_iters) / (lr_decay_iters - warmup_iters)
    assert 0 <= decay_ratio <= 1
    coeff = 0.5 * (1.0 + math.cos(math.pi * decay_ratio)) # coeff ranges 0..1
    return min_lr + coeff * (learning_rate - min_lr)

In [ ]:
t0 = time.time()
local_iter_num = 0 # number of iterations in the lifetime of this process

epochs = 3
model.train()
for epoch in range(epochs):
    print(f"Epoch [{epoch + 1}/{epochs}]")
    micro_step = 0
    for X, Y in train_loader:
        X = X.to(device)
        Y = Y.to(device)
        # determine and set the learning rate for this iteration
        lr = get_lr(iter_num) if decay_lr else learning_rate
        for param_group in optimizer.param_groups:
            param_group["lr"] = lr

        if iter_num % eval_interval == 0:
            val_loss = estimate_val_loss()
            print(f"step {iter_num}: val loss {val_loss:.4f}")
          
            if val_loss < best_val_loss or always_save_checkpoint:
                best_val_loss = val_loss
                if local_iter_num > 0:
                    checkpoint = {
                        "model": model.state_dict(),
                        "optimizer": optimizer.state_dict(),
                        "model_args": model_args,
                        "iter_num": iter_num,
                        "best_val_loss": best_val_loss
                    }
                    print(f"saving checkpoint to {out_dir}")
                    torch.save(checkpoint, os.path.join(out_dir, "ckpt.pt"))
        if local_iter_num == 0 and eval_only:
            break

        # forward backward update, with optional gradient accumulation to simulate larger batch size
        # and using the GradScaler if data type is float16

        with ctx:
            logits, loss = model(X, Y)
            loss = loss / gradient_accumulation_steps # scale the loss to account for gradient accumulation

        # backward pass, with gradient scaling if training in fp16
        scaler.scale(loss).backward()

        micro_step += 1
        if micro_step >= gradient_accumulation_steps:
            micro_step = 0
            # clip the gradient
            if grad_clip != 0.0:
                scaler.unscale_(optimizer)
                torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
            # step the optimizer and scaler if training in fp16
            scaler.step(optimizer)
            scaler.update()
            # flush the gradients as soon as we can, no need for this memory anymore
            optimizer.zero_grad(set_to_none=True)

            t1 = time.time()
            dt = t1 - t0
            t0 = t1
            if iter_num % log_interval == 0:
                # scale up to undo the division above, approximating the true total loss (exact would have been a sum)
                lossf = loss.item() * gradient_accumulation_steps
                print(f"iter {iter_num}: loss {lossf:.4f}, time {dt*1000:.2f}ms")
        
        iter_num += 1
        local_iter_num += 1
    
        if iter_num > max_iters:
            break

    if iter_num > max_iters:
            break